In [ ]:
# Authors: Isabella Silva Ferreira and Micael Veríssimo de Araújo
# e-mails: (is_bella21@hotmail.com)    (micaelvero@hotmail.com)


In [2]:
# Read data File and sapare the all the sets
import os
import pickle
import numpy as np
import time

from sklearn.decomposition import PCA
from sklearn.externals import joblib

print "All Right!"

All Right!


In [3]:
# Confirm our array shape
data = np.load("/home/micael/MyWorkspace/DATA/lvbb125Array.npy")
data_target = data[:,12]

data_inputs = data[:,:12]
print data.shape
data

(191100, 13)


array([[  9.52196307e-03,   6.12077856e-05,   1.65209174e-02, ...,
          9.21870469e+01,   5.98104895e+01,   6.10195977e+01],
       [  1.67018827e-02,   1.13179535e-01,   9.47747454e-02, ...,
          1.07602320e+02,   5.98104895e+01,   6.44654687e+01],
       [  8.95111356e-03,   9.14592529e-05,   8.82276613e-03, ...,
          3.70010500e+02,   4.97765697e+01,   1.06724656e+02],
       ..., 
       [  1.62160210e-02,   5.48769441e-03,   2.28184238e-02, ...,
          3.86259570e+01,   3.57536859e+01,   3.39022539e+01],
       [  1.42509257e-02,   1.21706940e-01,   4.58930284e-02, ...,
          1.07292875e+02,   5.50991584e+01,   9.68779375e+01],
       [  1.58070903e-02,   4.79239076e-02,   1.20003656e-01, ...,
          6.82103906e+01,   5.50991584e+01,   5.35117266e+01]])

In [4]:
print data_inputs.shape
data_inputs

(191100, 12)


array([[  9.52196307e-03,   6.12077856e-05,   1.65209174e-02, ...,
         -1.83297706e+00,   9.21870469e+01,   5.98104895e+01],
       [  1.67018827e-02,   1.13179535e-01,   9.47747454e-02, ...,
          1.78017831e+00,   1.07602320e+02,   5.98104895e+01],
       [  8.95111356e-03,   9.14592529e-05,   8.82276613e-03, ...,
          5.18436611e-01,   3.70010500e+02,   4.97765697e+01],
       ..., 
       [  1.62160210e-02,   5.48769441e-03,   2.28184238e-02, ...,
         -2.27387905e+00,   3.86259570e+01,   3.57536859e+01],
       [  1.42509257e-02,   1.21706940e-01,   4.58930284e-02, ...,
         -1.46886933e+00,   1.07292875e+02,   5.50991584e+01],
       [  1.58070903e-02,   4.79239076e-02,   1.20003656e-01, ...,
         -3.02542996e+00,   6.82103906e+01,   5.50991584e+01]])

In [5]:
print data_target.shape
data_target

(191100,)


array([  61.01959766,   64.46546875,  106.72465625, ...,   33.90225391,
         96.8779375 ,   53.51172656])

In [24]:
# Train NN
#from Functions import LogFunctions as log
from sklearn import cross_validation
from sklearn import preprocessing

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD
import keras.callbacks as callbacks
from keras.utils import np_utils

from datetime import datetime  
from datetime import timedelta 
import random


# Train information 
n_folds = 2
n_inits = 1
norm = 'mapstd'

train_info = {}
train_info['n_folds'] = n_folds
train_info['n_inits'] = n_inits
train_info['norm'] = norm

trn_desc = {}

# Slip the train and test sets
test_id= random.sample(range(data_inputs.shape[0]),int(np.floor(0.1*data_target.shape[0])))
lista_id = np.asarray(range(0, data_inputs.shape[0]))
all_id = np.ndarray.tolist(lista_id)
train_id = list(set(lista_id) - set(test_id))

                           

for ifold in range(0,4):
#    train_id, test_id = CVO[ifold]
    
    # normalize data based in train set
    if train_info['norm'] == 'mapstd':
        scaler = preprocessing.StandardScaler().fit(data_inputs[train_id,:])
    elif train_info['norm'] == 'mapstd_rob':
        scaler = preprocessing.RobustScaler().fit(data_inputs[train_id,:])
    elif train_info['norm'] == 'mapminmax':
        scaler = preprocessing.MinMaxScaler().fit(data_inputs[train_id,:])
        
    norm_all_data = scaler.transform(data_inputs)
    
# Normalize the outputs
    [freq, target_values] = np.histogram(data_target[train_id],
             bins=np.linspace(data_target[train_id].min(),data_target[train_id].max(),50))

    MOP = target_values[np.argmax(freq)]
    
    norm_target = data_target/MOP

    print 'Train Process for %i Fold'%(ifold)
    
    best_init = 0
    best_loss = 999
    
    for i_init in range(0,2):
        #print 'Init: %i of %i'%(i_init,train_info['n_inits'])
        model = Sequential()
        model.add(Dense(30, input_dim=data_inputs.shape[1], init='uniform'))
        model.add(Activation('tanh'))
        model.add(Dense(1, init='uniform')) 
        model.add(Activation('linear'))
        
        sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
        model.compile(loss='mean_squared_error', optimizer=sgd
                      ,metrics=['accuracy'])

        # Train model
        earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=25, 
                                            verbose=0, mode='auto')
        start_time = time.time()
        init_trn_desc = model.fit(norm_all_data[train_id], norm_target[train_id], 
                                nb_epoch=50, 
                                batch_size=8,
                                callbacks=[earlyStopping], 
                                verbose=0,
                                validation_data=(data_inputs[test_id],norm_target[test_id]),
                                shuffle=True)

        
    
    trn_desc[ifold] = init_trn_desc  


print "All Right!"

Train Process for 0 Fold
Train Process for 1 Fold
Train Process for 2 Fold
Train Process for 3 Fold
All Right!


In [30]:
from keras.models import load_model
output = "/home/micael/MyWorkspace/DATA/NN_outputs"

model.save(output+"/"+"Model1.h5")

joblib.dump(init_trn_desc.history,output+"/"+"Train_desc.jbl",compress=9)
joblib.dump([train_info],output+"/"+"Train_info.jbl",compress=9)

['/home/micael/MyWorkspace/DATA/NN_outputs/Train_info.jbl']

In [27]:
Teste = joblib.load(output+"/"+"Train_info.jbl")
Teste


[{'n_folds': 2, 'n_inits': 1, 'norm': 'mapstd'}]

In [31]:
print(init_trn_desc.history)

{'acc': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], 'loss': [0.06280715697716753, 0.059282822944832524, 0.058484474612448455, 0.058410033121236875, 0.058184519095528381, 0.05799509329597112, 0.057865393967317082, 0.057661485637893521, 0.057676429490932601, 0.057586225372488371, 0.057503920497924098, 0.057451582000720765, 0.057264933163296693, 0.057030318997242285, 0.057032671543973734, 0.056977243518412525, 0.056868065438363868, 0.056835467697274336, 0.056672522487848022, 0.056543808111263465, 0.056622762342623506, 0.056476838831101599, 0.056542573955981772, 0.056351498269618792, 0.056338175330502747, 0.056289363722553824, 0.056344795080813803, 0.056283969973294164, 0.056078448933172612, 0.056122430671194663, 0.056054691061273233, 0.056046496932267278, 0.056006241779284587, 0.056034959283253361, 